In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
print('Dependencies Imported')

In [ ]:
import pandas as pd

data = pd.read_csv(r"Coursera.csv")
print(data.head())


In [ ]:
data.shape


In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data['Difficulty Level'].value_counts()

In [ ]:
data['Course Rating'].value_counts()

In [ ]:
data['University'].value_counts()

In [ ]:
data['Course Name']

In [49]:
data = data[['Course Name','Difficulty Level','Course Description','Skills']]

In [ ]:
data.head(5)

In [51]:
'''Data Pre-Processing¶
An important part of the process is to pre-process the data into usable format for the recommendation system'''
# Removing spaces between the words (Lambda funtions can be used as well)

data['Course Name'] = data['Course Name'].str.replace(' ',',')
data['Course Name'] = data['Course Name'].str.replace(',,',',')
data['Course Name'] = data['Course Name'].str.replace(':','')
data['Course Description'] = data['Course Description'].str.replace(' ',',')
data['Course Description'] = data['Course Description'].str.replace(',,',',')
data['Course Description'] = data['Course Description'].str.replace('_','')
data['Course Description'] = data['Course Description'].str.replace(':','')
data['Course Description'] = data['Course Description'].str.replace('(','')
data['Course Description'] = data['Course Description'].str.replace(')','')

#removing paranthesis from skills columns
data['Skills'] = data['Skills'].str.replace('(','')
data['Skills'] = data['Skills'].str.replace(')','')

In [ ]:
data.head(5)

In [53]:
data['tags'] = data['Course Name'] + data['Difficulty Level'] + data['Course Description'] + data['Skills']

In [ ]:
data.head(5)

In [ ]:
data['tags'].iloc[1]

In [56]:
new_df = data[['Course Name','tags']]

In [ ]:
new_df.head(5)

In [ ]:
new_df['tags'] = data['tags'].str.replace(',',' ')

In [ ]:
new_df['Course Name'] = data['Course Name'].str.replace(',',' ')

In [ ]:
new_df.rename(columns = {'Course Name':'course_name'}, inplace = True)

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower()) #lower casing the tags column

In [ ]:
new_df.head(5)

In [ ]:
new_df.shape #3522 courses with tags and 2 columns (course_name and tags)

In [64]:
from sklearn.feature_extraction.text import CountVectorizer

In [65]:
cv = CountVectorizer(max_features=5000,stop_words='english')

In [66]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [67]:
import nltk #for stemming process

In [68]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [69]:
#defining the stemming function
def stem(text):
    y=[]

    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem) #applying stemming on the tags column

In [71]:
from sklearn.metrics.pairwise import cosine_similarity

In [72]:
similarity = cosine_similarity(vectors)

In [73]:
def recommend(course):
    course_index = new_df[new_df['course_name'] == course].index[0]
    distances = similarity[course_index]
    course_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:7]

    for i in course_list:
        print(new_df.iloc[i[0]].course_name)

In [79]:
def recommend(course_name):
    try:
        # Find index of the course in dataset
        idx = data[data['Course Name'].str.contains(course_name, case=False, na=False)].index[0]

        # Get similarity scores and sort them
        similar_courses = sorted(list(enumerate(similarity[idx])), key=lambda x: x[1], reverse=True)[1:6]

        print(f"\nTop 5 Course Recommendations for: {course_name}\n")
        for i, (index, score) in enumerate(similar_courses, 1):
            print(f"{i}. {data.iloc[index]['Course Name']} - Rating: {data.iloc[index]['Difficulty Level']}")

    except IndexError:
        print("Course not found. Please check the name and try again.")
